In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import graphviz
import pandas_profiling
import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix, precision_recall_fscore_support

from matplotlib.legend_handler import HandlerLine2D

%matplotlib inline

Columns:
* age - age in years
* sex - (1 = male; 0 = female)
* cp - chest pain type
* trestbps - resting blood pressure (in mm Hg on admission to the hospital)
* chol - serum cholestoral in mg/dl
* fbs - (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
* restecg - resting electrocardiographic results
* thalach - maximum heart rate achieved
* exang - exercise induced angina (1 = yes; 0 = no)
* oldpeak - ST depression induced by exercise relative to rest
* slope - the slope of the peak exercise ST segment
* ca - number of major vessels (0-3) colored by flourosopy
* thal - 3 = normal; 6 = fixed defect; 7 = reversable defect
* target - 1 or 0

In [ ]:
path = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
columns = ["age", "sex", "cp", 
           "trestbps", "chol", "fbs", 
           "restecg", "thalach","exang", 
           "oldpeak","slope", "ca", "thal", "num"]

data_pd = pd.read_csv(path, header=None, names=columns, na_values=['?'])
data_pd['target'] = np.where(data_pd.num > 0, 1, 0)
data_pd.drop('num', inplace=True, axis=1)

data_pd = data_pd.dropna() # 4 records

In [ ]:
pandas_profiling.ProfileReport(data_pd)

### Logistic Regression

Basic Training (train/test split)

In [ ]:
X = data_pd[data_pd.columns[:-1]]
y = data_pd['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

Cross validation

Grid search

### Metrics

Accuracy

Precision and Recall

Confusion Matrix

Roc / Precision-Recall Curve

Regularization (L1, L2)

### Logistic Regression Interpretation

Coefficients

Visualization

### Decision Tree

In [ ]:
def get_auc(dt, X, y):
    y_pred = dt.predict(X)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y, y_pred)
    return auc(false_positive_rate, true_positive_rate)

def plot_parameter_search(X_train, y_train, X_test, y_test, param_values, param_name):
    train_results = []
    test_results = []

    for val in param_values:
        dt = tree.DecisionTreeClassifier(**{param_name: val})
        dt.fit(X_train, y_train)

        train_results.append(get_auc(dt, X_train, y_train))
        test_results.append(get_auc(dt, X_test, y_test))

    line1, = plt.plot(param_values, train_results, 'b', label='Train AUC')
    line2, = plt.plot(param_values, test_results, 'r', label='Test AUC')
    plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
    plt.ylabel('AUC score')
    plt.xlabel(param_name)
    plt.show()


parameters = {
    'max_depth': np.linspace(1, 10, 10, endpoint=True),
    'min_samples_leaf': np.linspace(0.01, 0.5, 20, endpoint=True),
    'min_samples_split': np.linspace(0.0001, 1.0, 20, endpoint=True),
    'max_features': list(range(1,X_train.shape[1])),
    
}
    
for param, values in parameters.items():
    plot_parameter_search(X_train, y_train, X_test, y_test, values, param)

In [ ]:
max_features = list(range(1,X_train.shape[1]))

train_results = []
test_results = []

for val in max_features:
    dt = tree.DecisionTreeClassifier(**{'max_features': val, 'max_depth': 4})
    dt.fit(X_train, y_train)

    train_results.append(get_auc(dt, X_train, y_train))
    test_results.append(get_auc(dt, X_test, y_test))

line1, = plt.plot(max_features, train_results, 'b', label='Train AUC')
line2, = plt.plot(max_features, test_results, 'r', label='Test AUC')
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC score')
plt.xlabel('max_features (max_depth=4)')
plt.show()

Grid Search

In [ ]:
del parameters['min_samples_split']

grid_search = GridSearchCV(tree.DecisionTreeClassifier(), 
                   parameters, 
                   cv=5, 
                   scoring='recall', 
                   refit=True,
                   return_train_score=True, 
                   n_jobs=-1)
grid_search.fit(X_train, y_train)

print(grid_search.best_score_)
print(grid_search.best_params_)

clf = grid_search.best_estimator_

Precision/Recall

In [ ]:
y_pred = clf.predict(X_train)

precision_recall_fscore_support(y_train, y_pred, average='binary')

In [ ]:
y_pred = clf.predict(X_test)

precision_recall_fscore_support(y_test, y_pred, average='binary')

### Decision Tree Interpretation

Feature importance

In [ ]:
sns.barplot(x=clf.feature_importances_, y=data_pd.columns[:-1])

Visualization

In [ ]:
dot_data = tree.export_graphviz(clf, feature_names=data_pd.columns[:-1],  
                                out_file=None, class_names=True,
                                filled=True, rounded=True)  
graphviz.Source(dot_data)  

Decision tree boundaries

More advanced interpretation techniques: